In [ ]:
import pandas as pd
import numpy as np
import cv2
import os

# -------------------------------
# 1. Load merged dataset
# -------------------------------
df = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2026-analytics-1/merged_cleaned_features.csv", low_memory=False)

# -------------------------------
# 2. Video parameters
# -------------------------------
fps = 10
width, height = 1200, 800
output_video = "/kaggle/working/nfl_synthetic.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# -------------------------------
# 3. Normalize coordinates
# -------------------------------
df['x_pixel'] = (df['x_input'] * (width/120)).astype(int)
df['y_pixel'] = (df['y_input'] * (height/53.3)).astype(int)

# -------------------------------
# 4. Loop per frame
# -------------------------------
for frame_id in sorted(df['frame_id'].unique()):
    frame = np.zeros((height, width, 3), dtype=np.uint8)
    frame[:] = (34, 139, 34)  # green RGB

    df_frame = df[df['frame_id'] == frame_id]

    for _, row in df_frame.iterrows():
        x = row['x_pixel']
        y = row['y_pixel']
        side = row['player_side']

        color = (0,0,255) if side == 'home' else (255,0,0)
        cv2.circle(frame, (x, y), 5, color, -1)

        cv2.putText(frame, str(row['nfl_id']), (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1)

    out.write(frame)

# -------------------------------
# 5. Release video
# -------------------------------
out.release()
print("✅ Synthetic NFL overlay video created:", output_video)
